In [20]:
import pandas as pd
import numpy as np

import pyodbc
import os
from tqdm import tqdm

In [21]:
CONNECTION_STRING = "Enter Connection String Here"

In [287]:
def read_csv(file, file_type, Transect, Recorder_Person, Date, Time, Latitude, Longitude, Depth, COG, Coverage):
    """ Function reads csv and creates DF with the above columns. Parameters used to define column names."""
    def clean_columns(column):
        """Takes columns and rounds them 6 decimal places and makes float"""
        # if type datetime convert to float
        
        return round(float(str(column)), 6)
    if file_type == 'csv':
        raw_data = pd.read_csv(file)
    elif file_type == 'excel':
        raw_data = pd.read_excel(file)
    df = pd.DataFrame(raw_data, columns=["Transect", "Recorder", "Datetime", "Latitude", "Longitude", "Depth", "COG", "SliderCoverage"])
    df['Transect'] = raw_data[Transect]
    df['Recorder'] = Recorder_Person
    df['Latitude'] = raw_data[Latitude]
    df['Longitude'] = raw_data[Longitude]
    df['Depth'] = raw_data[Depth]
    if COG == None:
        df['COG'] = None
    else:
        raw_data[COG] = raw_data[COG].astype(str)
        df['COG'] = raw_data[COG]
        df['COG'] = df['COG'].apply(clean_columns)
    df['SliderCoverage'] = raw_data[Coverage]
    
    df['Latitude'] = df['Latitude'].apply(clean_columns)
    df['Longitude'] = df['Longitude'].apply(clean_columns)
    df['Depth'] = df['Depth'].apply(clean_columns)
    # drop empty rows in slider coverage
    df = df.dropna(subset=['SliderCoverage'])

    # convert date and time to str
    raw_data[Date] = raw_data[Date].astype(str)
    raw_data[Time] = raw_data[Time].astype(str)

    # combine date and time

    df['Datetime'] = raw_data[Date] + ' ' + raw_data[Time]
    return df

In [288]:
def create_table(transect, year, type):
    """ Function creates table in SQL DB with the name of the transect and year. """
    transect = transect
    year = year
    table_name = transect + '_' + year + '_' + type
    conn = pyodbc.connect(CONNECTION_STRING)
    cursor = conn.cursor()
    cursor.execute(
        f"""CREATE TABLE [dbo].[{table_name}] (
        [TransectId]     VARCHAR (50) NULL,
        [Recorder]       VARCHAR (10) NULL,
        [Datetime]       DATETIME     NULL,
        [Longitude]      FLOAT (53)   NULL,
        [Latitude]       FLOAT (53)   NULL,
        [Depth]          FLOAT (53)   NULL,
        [COG]            INT          NULL,
        [SliderCoverage] INT          NULL
        );
        """
        )
    conn.commit()
    conn.close()
    print(f"Table {table_name} created.")

In [289]:
def upload_data(df, table_name):
    """ Function uploads DF to SQL table. """
    with pyodbc.connect(CONNECTION_STRING) as conn:
        cursor = conn.cursor()
        for index, row in tqdm(df.iterrows(), total=df.shape[0]):
            try: 
                cursor.execute("INSERT INTO " + table_name + " (TransectId, Recorder, Datetime, Latitude, Longitude, Depth, COG, SliderCoverage) values(?,?,?,?,?,?,?,?)", row['Transect'], row['Recorder'], row['Datetime'], row['Latitude'], row['Longitude'], row['Depth'], row['COG'], row['SliderCoverage'])
            except Exception as e:            
                print(f"Error: {e}", row['Transect'], row['Recorder'], row['Datetime'], row['Latitude'], row['Longitude'], row['Depth'], row['COG'], row['SliderCoverage'])
            conn.commit()
        cursor.close()
    print("Data successfully uploaded.")

